# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()
experiment_name = 'website_classification_hyperdrive'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
website_classification_hyperdrive,quick-starts-ws-138723,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-hd"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded..................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Website Classification Dataset"
description_text = "Website Classification DataSet for Capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Panth-Shah/nd00333-capstone/master/Dataset/malicious_website_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [6]:
df = dataset.to_pandas_dataframe()
df.describe()

,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,TCP_CONVERSATION_EXCHANGE,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
count,1781.000000,1781.000000,1781.000000,1781.000000,1781.000000,1.781000e+03,1781.000000,1781.000000,1.781000e+03,1.781000e+03,1781.000000,1780.000000,1781.000000
mean,56.961258,11.111735,16.261089,5.472768,3.060640,2.982339e+03,18.540146,18.746210,1.589255e+04,3.155599e+03,18.540146,2.263483,0.121280
std,27.555586,4.549896,40.500975,21.807327,3.386975,5.605057e+04,41.627173,46.397969,6.986193e+04,5.605378e+04,41.627173,2.930853,0.326544
min,16.000000,5.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,39.000000,8.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
50%,49.000000,10.000000,7.000000,0.000000,2.000000,6.720000e+02,8.000000,9.000000,5.790000e+02,7.350000e+02,8.000000,0.000000,0.000000
75%,68.000000,13.000000,22.000000,5.000000,5.000000,2.328000e+03,26.000000,25.000000,9.806000e+03,2.701000e+03,26.000000,4.000000,0.000000
max,249.000000,43.000000,1194.000000,708.000000,17.000000,2.362906e+06,1198.000000,1284.000000,2.060012e+06,2.362906e+06,1198.000000,20.000000,1.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

In [9]:
# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.01, 100), 
        "max_iter": choice(16, 32, 64, 128, 256)
    }
)

### Create an Environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [10]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [11]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [12]:
if "training" not in os.listdir():
    os.mkdir("./training")

from azureml.core import ScriptRunConfig
import os

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(
    source_directory=os.path.join('./'), 
    compute_target=amlcompute_cluster_name, 
    script='train.py', 
    environment=sklearn_env
)

In [13]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=4)

### Submit your experiment

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_98ace3bb-e35a-40d7-8e94-d9acb494dc69
Web View: https://ml.azure.com/experiments/website_classification_hyperdrive/runs/HD_98ace3bb-e35a-40d7-8e94-d9acb494dc69?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-138723/workspaces/quick-starts-ws-138723

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-12T18:59:15.313784][API][INFO]Experiment created<END>\n""<START>[2021-02-12T18:59:15.883082][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-02-12T18:59:16.043334][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-02-12T18:59:16.5197657Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
#Get best run for hyperdrive

get_best_performance = hyperdrive_run.get_best_run_by_primary_metric() 
get_best_metrics = get_best_performance.get_metrics() 
print('Best Run Id: ', get_best_performance.id)
print('Best Run Accuracy:', get_best_metrics['Accuracy'])
print('Best Run Regularization Strength:', get_best_metrics['Regularization Strength:']) 
print('Best Run Max iterations:', get_best_metrics['Max iterations:'])

Best Run Id:  HD_899ea202-037e-4a5a-b21a-632e0a341bc0_9
Best Run Accuracy: 0.9607843137254902
Best Run Regularization Strength: 88.1908640736605
Best Run Max iterations: 64


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [15]:
get_best_performance.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_581246a5541724a1802b0ecf35023e05503c81438863382370a4383125744c7f_d.txt',
 'azureml-logs/65_job_prep-tvmps_581246a5541724a1802b0ecf35023e05503c81438863382370a4383125744c7f_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_581246a5541724a1802b0ecf35023e05503c81438863382370a4383125744c7f_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/BestAutoMLRun_Properties.txt',
 'logs/azureml/104_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/best_model_autoML.pkl',
 'outputs/best_model_hyperdrive.joblib',
 'outputs/capstone_model.joblib',
 'outputs/envFile.yml',
 'outputs/score.py']

In [16]:
#TODO: Save the best model

import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
get_best_performance.register_model(model_name = 'best_model', model_path = 'outputs/capstone_model.joblib')

Model(workspace=Workspace.create(name='quick-starts-ws-138308', subscription_id='b968fb36-f06a-4c76-a15f-afab68ae7667', resource_group='aml-quickstarts-138308'), name=best_model, id=best_model:1, version=1, tags={}, properties={})

In [17]:
get_best_performance.download_file("outputs/capstone_model.joblib","outputs/best_model_hyperdrive.joblib")

In [18]:
get_best_performance

Experiment,Id,Type,Status,Details Page,Docs Page
website_classification_hyperdrive,HD_899ea202-037e-4a5a-b21a-632e0a341bc0_9,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [21]:
# Get your best run and save the model from that run.

model = get_best_performance.register_model(model_name = 'best_Hyperdrive_model', model_path =  'outputs/capstone_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
# Download scoring file 
get_best_performance.download_file('outputs/score.py', 'outputs/hyperdrive_score.py')
script_file_name = 'outputs/hyperdrive_score.py'

In [24]:
# Download environment file
get_best_performance.download_file('outputs/envFile.yml', 'outputs/envFile.yml')

TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'area': "bmData", 'type': "capstone_autoML_Classifier"}, 
                                               description = 'sample service for Capstone Project AutoML Classifier for Websites')

In [ ]:
# deploy
aci_service_name = 'capstone-hyperdrive-deploy'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)

capstone-hyperdrive-deploy
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................

In [ ]:
print(aci_service.state)
print(aci_service.scoring_uri)
print(aci_service.swagger_uri)